### Проект по SQL

### Доступ к базе данных


In [1]:
import pandas as pd
import sqlalchemy as sa

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)


### Исследование таблиц

In [3]:
# функция для просмотра основной информации о таблицах
def analyze_df(df):
    print('5 случайных строк датафрейма')
    display(df.sample(5))
    print('---------------------------------------------------------------------------------------------------------')
    print('Информации о датафрейме')
    display(df.info())
 

В таблице books 1000 строк, в  authors 636 строк, publishers 340 строк , ratings 6456 строк , reviews 2793 строк.

### Задачи

#### Cколько книг вышло после 1 января 2000 года


In [5]:
query = '''
            SELECT 
              COUNT(*) 
            FROM 
              books 
            WHERE 
              publication_date > '2000-01-01'
         '''

get_sql_data(query)

,count
0,819


После 1 января 2000 года вышло 819 книг.

#### Для каждой книги  количество обзоров и средняя оценка

In [13]:
query = ''' 
            WITH reviews_count AS (
              SELECT 
                book_id, 
                COUNT(username) AS count_reviews 
              FROM 
                reviews 
              GROUP BY 
                book_id
            ), 
            average_rating AS (
              SELECT 
                book_id, 
                ROUND(
                  AVG(rating), 
                  2
                ) AS avg_ratings 
              FROM 
                ratings 
              GROUP BY 
                book_id
            ) 
            SELECT 
              b.title, 
              COALESCE(rc.count_reviews, 0) AS count_reviews, 
              COALESCE(ar.avg_ratings, 0) AS avg_ratings 
            FROM 
              books AS b 
              LEFT JOIN reviews_count AS rc USING (book_id) 
              LEFT JOIN average_rating AS ar USING (book_id)
            ORDER BY
                avg_ratings DESC
        '''

get_sql_data(query)

,title,count_reviews,avg_ratings
0,Pop Goes the Weasel (Alex Cross #5),2,5.00
1,The Ghost Map: The Story of London's Most Terr...,2,5.00
2,In the Hand of the Goddess (Song of the Liones...,2,5.00
3,Tai-Pan (Asian Saga #2),2,5.00
4,How to Be a Domestic Goddess: Baking and the A...,1,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Junky,2,2.00
997,His Excellency: George Washington,2,2.00
998,Drowning Ruth,3,2.00


In [7]:
query = '''
            WITH reviews_count AS (
              SELECT 
                book_id, 
                COUNT(username) AS count_reviews 
              FROM 
                reviews 
              GROUP BY 
                book_id
            ), 
            average_rating AS (
              SELECT 
                book_id, 
                ROUND(
                  AVG(rating), 
                  2
                ) AS avg_ratings 
              FROM 
                ratings 
              GROUP BY 
                book_id
            ), 
            books_with_reviews AS (
              SELECT 
                b.title, 
                COALESCE(rc.count_reviews, 0) AS count_reviews, 
                COALESCE(ar.avg_ratings, 0) AS avg_ratings 
              FROM 
                books AS b 
                LEFT JOIN reviews_count AS rc USING (book_id) 
                LEFT JOIN average_rating AS ar USING (book_id)
            ) 
            SELECT 
              count_reviews, 
              COUNT(*) AS number_of_books 
            FROM 
              books_with_reviews 
            GROUP BY 
              count_reviews 
            ORDER BY 
              number_of_books DESC

        '''

get_sql_data(query)



,count_reviews,number_of_books
0,2,444
1,3,277
2,4,160
3,5,60
4,1,37
5,6,15
6,0,6
7,7,1


In [8]:
query = '''
            WITH reviews_count AS (
                SELECT 
                    book_id, 
                    COUNT(username) AS count_reviews 
                FROM 
                    reviews 
                GROUP BY 
                    book_id
            ), 
            average_rating AS (
                SELECT 
                    book_id, 
                    ROUND(AVG(rating), 2) AS avg_ratings 
                FROM 
                    ratings 
                GROUP BY 
                    book_id
            ), 
            books_with_reviews AS (
                SELECT 
                    b.title, 
                    COALESCE(rc.count_reviews, 0) AS count_reviews, 
                    COALESCE(ar.avg_ratings, 0) AS avg_ratings 
                FROM 
                    books AS b 
                    LEFT JOIN reviews_count AS rc USING (book_id) 
                    LEFT JOIN average_rating AS ar USING (book_id)
            ) 
            SELECT 
                title, 
                count_reviews, 
                avg_ratings 
            FROM 
                books_with_reviews 
            WHERE 
                avg_ratings = 5
        '''

get_sql_data(query)

,title,count_reviews,avg_ratings
0,Light in August,2,5.0
1,Act of Treason (Mitch Rapp #9),2,5.0
2,The Demon-Haunted World: Science as a Candle i...,2,5.0
3,Dead Souls,2,5.0
4,March,2,5.0
5,Wherever You Go There You Are: Mindfulness Me...,2,5.0
6,Neil Gaiman's Neverwhere,2,5.0
7,Evening Class,2,5.0
8,The Ghost Map: The Story of London's Most Terr...,2,5.0
9,Welcome to Temptation (Dempseys #1),2,5.0


Максимальное количество обзоров, которое было предоставлено на одну книгу, составило 7. Есть также 6 книг, которые не были удостоены ни одного обзора. В основном, большинство книг имеют 2 обзора. Средний рейтинг 5 имеют 44 книги.

#### Издательство, которое выпустило наибольшее число книг толще 50 страниц — так  исключите из анализа брошюры


In [9]:
query = '''
            SELECT 
              p.publisher, 
              COUNT(b.book_id) AS book_count 
            FROM 
              books b 
              JOIN publishers p USING (publisher_id)
            WHERE 
              b.num_pages > 50 
            GROUP BY 
              p.publisher 
            ORDER BY 
              book_count DESC 
            LIMIT 
              1
        '''

get_sql_data(query)

,publisher,book_count
0,Penguin Books,42


Издательство, которое выпустило наибольшее число книг толще 50 страниц — Penguin Books они издали 42 книги

#### Автор с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками


In [10]:
query = '''
            WITH book_ratings AS (
              SELECT 
                book_id, 
                AVG(rating) AS avg_rating, 
                COUNT(rating) AS rating_count 
              FROM 
                ratings 
              GROUP BY 
                book_id
            ), 
            filtered_books AS (
              SELECT 
                br.book_id, 
                br.avg_rating, 
                br.rating_count 
              FROM 
                book_ratings br 
              WHERE 
                br.rating_count >= 50
            ), 
            author_avg_ratings AS (
              SELECT 
                b.author_id, 
                AVG(fb.avg_rating) AS author_avg_rating, 
                SUM(fb.rating_count) AS total_ratings 
              FROM 
                filtered_books fb 
                JOIN books b  USING (book_id)
              GROUP BY 
                b.author_id
            ) 
            SELECT 
              a.author, 
              aar.author_avg_rating, 
              aar.total_ratings 
            FROM 
              author_avg_ratings aar 
              JOIN authors a USING (author_id)
            ORDER BY 
              aar.author_avg_rating DESC 
            LIMIT 
              1
        '''
get_sql_data(query)

,author,author_avg_rating,total_ratings
0,J.K. Rowling/Mary GrandPré,4.283844,310.0


Автор с самой высокой средней оценкой книг, учитывая только произведения, которые получили более 50 оценок, — это J.K. Rowling/Mary GrandPré. Её средняя оценка составляет 4.28, общее количество оценок для всех книг автора 310

#### Среднее количество обзоров от пользователей, которые поставили больше 48 оценок.


In [11]:
query = '''
            WITH user_ratings AS (
              SELECT 
                username, 
                COUNT(rating_id) AS rating_count 
              FROM 
                ratings 
              GROUP BY 
                username 
              HAVING 
                COUNT(rating_id) > 48
            ), 
            user_reviews AS (
              SELECT 
                r.username, 
                COUNT(r.review_id) AS review_count 
              FROM 
                reviews r 
                JOIN user_ratings ur  USING (username)
              GROUP BY 
                r.username
            ) 
            SELECT 
              AVG(review_count) AS average_reviews 
            FROM 
              user_reviews

'''
get_sql_data(query)


,average_reviews
0,24.0


Среднее количество обзоров от пользователей, которые поставили более 48 оценок, составляет 24.

### Выводы 

*Выводы по задачам:*

• После 1 января 2000 года вышло 819 книг.

• Максимальное количество обзоров, которое было предоставлено на одну книгу, составило 7. Есть также 6 книг, которые не были удостоены ни одного обзора. В основном, большинство книг имеют 2 обзора. Средний рейтинг 5 имеют 44 книги.  

• Издательство, которое выпустило наибольшее число книг толще 50 страниц — Penguin Books они издали 42 книги. 

• Автор с самой высокой средней оценкой книг, учитывая только произведения, которые получили более 50 оценок, — это J.K. Rowling/Mary GrandPré. Её средняя оценка составляет 4.28, общее количество оценок для всех книг автора 310. 

• Среднее количество обзоров от пользователей, которые поставили более 48 оценок, составляет 24.

*Рекомендации:*

Используя данные о книгах с высоким количеством обзоров и высокими рейтингами, можно создать систему рекомендаций, которая будет предлагать пользователям книги, которые уже получили положительные отзывы от других читателей. Например, книги от J.K. Rowling/Mary GrandPré могут быть рекомендованы пользователям, которые ищут качественную литературу. Учитывая, что Penguin Books выпустило наибольшее количество книг, можно создать специальный раздел , посвященную книгам этого издательства. 